In [46]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
pd.set_option('display.max_rows', 500)  # let pd.df display() 500 rows
pd.set_option('display.max_columns', 500)

### Import data

In [35]:
raw = pd.read_csv("../clean_data/df_3to1.csv")
raw.drop(["TAU","ABETA","PTAU"], axis=1, inplace=True)

/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_65008/837559297.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv("../clean_data/df_3to1.csv")


In [63]:
# temp 
adni1 = raw.loc[raw["ORIGPROT"]==0]
start_mci = adni1["RID"][(adni1["VISCODE"]==0) & (adni1["DX"]==2)]
adni1_mci = adni1.loc[(adni1["VISCODE"].isin([0,12]))&         # keep time point 0,12,24
            (adni1["RID"].isin(start_mci))             # rm start with AD
           ] 
adni1_mci.DX.value_counts()
# turn to binary problem:
adni1_mci["DX"].replace()

2    683
3     64
1      7
Name: DX, dtype: int64

### Missing data

In [52]:
display(adni1.isna().mean().sort_values(ascending=False))

FBB                      0.999806
PIB                      0.956968
AV45                     0.902888
TAU_UPENN                0.797829
PTAU_UPENN               0.797054
ABETA_UPENN              0.797054
EcogPtOrgan              0.771274
EcogPtVisspat            0.768560
EcogSPOrgan              0.768172
EcogPtDivatt             0.768172
EcogSPDivatt             0.767978
EcogPtPlan               0.765846
EcogSPVisspat            0.764683
EcogPtLang               0.764683
EcogPtTotal              0.764489
EcogPtMem                0.763908
EcogSPPlan               0.760225
MOCA                     0.759837
EcogSPTotal              0.756736
EcogSPMem                0.756348
EcogSPLang               0.756154
FDG                      0.594883
OTHER                    0.368482
DPMOOD                   0.367513
RASH                     0.367319
FALL                     0.367319
WANDER                   0.367319
ABDOMN                   0.367319
CRYING                   0.367319
INSOMN        